# Setup

In [1]:
import os

In [2]:
os.chdir('..')

In [3]:
os.getcwd()

'c:\\Users\\I571927\\PycharmProjects\\Nerf'

In [8]:
import yaml
import numpy as np
import imageio

set: fern, flower, fortress, norns, leaves, orchids, room, trex

In [98]:
dataset = 'flower'
datadir = f'./dataset/nerf_llff_data/{dataset}'
factor = 8
bd_factor=.75
path_zflat=False

# _load_data

src.load_llff에서 사용하는 pipeline을 refactoring하고 있는 것

크게 factor가 주어졌을 때와 그렇지 않을때를 구분해야할 것으로 보이고 factor가 주어졌을 때 진행되는 update는 다음과 같다.

- minify : 원본 이미지를 factor에 맞춰서 mogrify로 줄여주는 과정
- update_pose_bd : factor만큼 f를 조정하고, hw는 mogrify로 줄여진 크기를 사용

## _load_data (Phase 1)

I: 
- datadir
- factor: minify할 때 사용. 어떤 이미지를 불러올지 결정하는 것

O: 

### get_sh

원래 img0라고 하던 것을 img_path로 정정

In [39]:
img_path = [os.path.join(datadir, 'images', f) for f in sorted(os.listdir(os.path.join(datadir, 'images'))) \
        if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')][0]
sh = imageio.imread(img_path).shape

C:\Users\I571927\AppData\Local\Temp\ipykernel_21688\2227507593.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  sh = imageio.imread(img_path).shape


In [40]:
sh

(3024, 4032, 3)

## minify

이건 이미지의 가로, 세로, factor 크기를 명시했을 때 aspect ratio를 맞춰서 줄여주고 image*로 만든 폴더에 옮겨두는 것에 해당한다!

subprocess로 image를 resize를 하게 하며  mogrify를 사용해서 진행한다.

현재 데이터 셋에서는 이미 해당도가 맞춰서 정리되어 있기 때문에 구현하지 않도록 하고, 추후에 변경이 필요하다면 offline으로 처리하도록 한다

기존 코드에서는 factor만 알려주는 형태로 구현되어 있다. 

### chck_dir

이건 알려준 shape에 해당하는 이미지가 있는지 없는지 확인

I
- factor_list: [factor]로 단순히 list로 factor를 받은 것 <br>
O: 

In [42]:
factor_list = [factor]
resolutions = []

needtoload = False
for r in factor_list:
    imgdir = os.path.join(datadir, 'images_{}'.format(r))
    if not os.path.exists(imgdir):
        needtoload = True
for r in resolutions:
    imgdir = os.path.join(datadir, 'images_{}x{}'.format(r[1], r[0]))
    if not os.path.exists(imgdir):
        needtoload = True

### get_img_path

In [43]:
from shutil import copy
from subprocess import check_output

imgdir = os.path.join(datadir, 'images')
imgs = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir))]
imgs = [f for f in imgs if any([f.endswith(ex) for ex in ['JPG', 'jpg', 'png', 'jpeg', 'PNG']])]
imgdir_orig = imgdir

## _load_data(Phase 2)

### get_img_path

factor만 조정한다면 sfx는 _factor로 된 폴더를 사용한다.

In [50]:
sfx = f"_{factor}"

imgdir = os.path.join(datadir, 'images' + sfx)
img_paths = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir)) if
                f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]

### get_pose_bd

In [90]:
poses_arr = np.load(os.path.join(datadir, 'poses_bounds.npy'))
poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1, 2, 0])
bds = poses_arr[:, -2:].transpose([1, 0])

In [54]:
poses_arr.shape

(34, 17)

In [55]:
# (34, 17)->(34,15)->(34,3,5)->(3,5,34)
poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1, 2, 0]).shape

(3, 5, 34)

In [56]:
poses_arr[:, -2:].transpose([1, 0]).shape

(2, 34)

In [72]:
for i in range(5):
    print(i, poses[:,i,0])

0 [0.01294097 0.99962852 0.02398638]
1 [ 0.99991381 -0.01288403 -0.00252655]
2 [-0.00221657  0.02401701 -0.99970909]
3 [-5.55752123 -2.23216201  1.38032517]
4 [3024.         4032.         3575.05860595]


### Update_pose_bd

In [91]:
# hwf를 update하는 것으로 factor를 지정했을 때 업데이트 하게 된다.
sh = imageio.imread(img_paths[0]).shape
poses[:2, 4, :] = np.array(sh[:2]).reshape([2, 1])
poses[2, 4, :] = poses[2, 4, :] * 1. / factor

C:\Users\I571927\AppData\Local\Temp\ipykernel_21688\2947389849.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  sh = imageio.imread(img_paths[0]).shape


In [92]:
for i in range(5):
    print(i, poses[:,i,0])

0 [0.01294097 0.99962852 0.02398638]
1 [ 0.99991381 -0.01288403 -0.00252655]
2 [-0.00221657  0.02401701 -0.99970909]
3 [-5.55752123 -2.23216201  1.38032517]
4 [378.         504.         446.88232574]


### get_img

In [68]:
def imread(f):
    if f.endswith('png'):
        return imageio.imread(f, ignoregamma=True)
    else:
        return imageio.imread(f)

imgs = imgs = [imread(f)[..., :3] / 255. for f in img_paths]
imgs = np.stack(imgs, -1)

C:\Users\I571927\AppData\Local\Temp\ipykernel_21688\1962279215.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(f, ignoregamma=True)


# Update roation matrix

여기에서는 다시 (# image, xyz, # vector)로 재구성 됨

이렇게 한 이유는 pose가 [-y, x, z] 로 구성되어 있기 때문이다!!!

In [93]:
# Correct rotation matrix ordering and move variable dim to axis 0
# xyz * (ori1, ori0, ori2, ori3, hwf)
poses = np.concatenate([poses[:, 1:2, :], -poses[:, 0:1, :], poses[:, 2:, :]], 1)

In [94]:
for i in range(5):
    print(i, poses[:, i, 0])

0 [ 0.99991381 -0.01288403 -0.00252655]
1 [-0.01294097 -0.99962852 -0.02398638]
2 [-0.00221657  0.02401701 -0.99970909]
3 [-5.55752123 -2.23216201  1.38032517]
4 [378.         504.         446.88232574]


In [95]:
# 여기에서 # image를 맨 앞으로 옮기게 됨
poses = np.moveaxis(poses, -1, 0).astype(np.float32)
imgs = np.moveaxis(imgs, -1, 0).astype(np.float32)
images = imgs
bds = np.moveaxis(bds, -1, 0).astype(np.float32)

In [97]:
for i in range(5):
    print(i, poses[0, :, i])

0 [ 0.9999138  -0.01288403 -0.00252655]
1 [-0.01294096 -0.99962854 -0.02398638]
2 [-0.00221657  0.02401701 -0.99970907]
3 [-5.5575213 -2.232162   1.3803252]
4 [378.      504.      446.88232]


# Update bd

In [99]:
# Rescale if bd_factor is provided
sc = 1. if bd_factor is None else 1. / (bds.min() * bd_factor)
poses[:, :3, 3] *= sc
bds *= sc

In [100]:
for i in range(5):
    print(i, poses[0, :, i])

0 [ 0.9999138  -0.01288403 -0.00252655]
1 [-0.01294096 -0.99962854 -0.02398638]
2 [-0.00221657  0.02401701 -0.99970907]
3 [-0.32825887 -0.13184421  0.08152987]
4 [378.      504.      446.88232]


# Recenter

본 코드에서는 기본값으로 recenter를 하게 된다.

In [102]:
def normalize(x):
    return x / np.linalg.norm(x)

## get_viewmatrix

In [106]:
poses.shape

(34, 3, 5)

In [103]:
hwf = poses[0, :3, -1:]

center = poses[:, :3, 3].mean(0)
vec2 = normalize(poses[:, :3, 2].sum(0))
up = poses[:, :3, 1].sum(0)

z, up, pos = vec2, up, center

In [202]:
def get_viewmatrix(z, up, pos):
    vec2 = normalize(z)
    vec1_avg = up
    vec0 = normalize(np.cross(vec1_avg, vec2))
    vec1 = normalize(np.cross(vec2, vec0))
    m = np.stack([vec0, vec1, vec2, pos], 1)
    return m

In [113]:
vec2 = normalize(z)
vec1_avg = up
vec0 = normalize(np.cross(vec1_avg, vec2))
vec1 = normalize(np.cross(vec2, vec0))
viewmatrix = np.stack([vec0, vec1, vec2, pos], 1)

In [115]:
viewmatrix.shape

(3, 4)

In [128]:
viewmatrix[:, 0]

array([ 0.99990654,  0.00250978, -0.01343402], dtype=float32)

## get_c2w

In [145]:
c2w = np.concatenate([viewmatrix, hwf], 1)

In [118]:
c2w.shape

(3, 5)

## recenter_poses 최종

In [132]:
poses_ = poses + 0
bottom = np.reshape([0, 0, 0, 1.], [1, 4])
c2w = np.concatenate([c2w[:3, :4], bottom], -2)

In [122]:
c2w.shape

(4, 4)

In [129]:
c2w[:, 0]

array([ 0.99990654,  0.00250978, -0.01343402,  0.        ])

In [133]:
# 이미지에 대해서 각각 만들어 줌.
bottom = np.tile(np.reshape(bottom, [1, 1, 4]), [poses.shape[0], 1, 1])

In [134]:
bottom.shape

(34, 1, 4)

In [135]:
poses = np.concatenate([poses[:, :3, :4], bottom], -2)

In [136]:
poses.shape

(34, 4, 4)

카메라 모델
![img](./assets/pipeline-02-typical_perspective_model.PNG)

In [137]:
# 위의 이미지에서 M(3,4)가 기존의 c2w에서 view matrix의 역할이었던 것
poses = np.linalg.inv(c2w) @ poses

In [140]:
poses.shape

(34, 4, 4)

In [139]:
# 내가 보았을 때 recenter의 의미가 결과적으로 P에 M^{-1}을 행렬곱한 것을 poses로 사용하는 것 같다.
poses_[:, :3, :4] = poses[:, :3, :4]
poses = poses_

array([7.94865284e-03, 6.87575154e-03, 2.62866117e-04, 1.00000000e+00])

In [146]:
poses_ = poses + 0
bottom = np.reshape([0, 0, 0, 1.], [1, 4])
c2w = np.concatenate([c2w[:3, :4], bottom], -2)
bottom = np.tile(np.reshape(bottom, [1, 1, 4]), [poses.shape[0], 1, 1])
poses = np.concatenate([poses[:, :3, :4], bottom], -2)

poses = np.linalg.inv(c2w) @ poses
poses_[:, :3, :4] = poses[:, :3, :4]
poses = poses_

In [148]:
poses.shape

(34, 4, 4)

pose 바뀌기 전<br>
![img](./assets/eda-01-pose%20state%201.png)

In [151]:
# 여기에서는 hwf가 빠지게 된다.
for i in range(4):
    print(i, poses[0, :, i])

0 [ 0.99991585 -0.01279143 -0.00222709  0.        ]
1 [-0.01279741 -0.99991444 -0.00268797  0.        ]
2 [-0.00219251  0.00271624 -0.9999938   0.        ]
3 [-0.34415161 -0.1301463   0.08449194  1.        ]


# Get_render_poses

In [158]:
c2w = np.concatenate([viewmatrix, hwf], 1)

## Get spiral
# Get average pose
up = normalize(poses[:, :3, 1].sum(0))

up

array([ 0.0025097 , -0.99994036,  0.01062915])

## Get focus depth

In [167]:
# Find a reasonable "focus depth" for this dataset
# 이건 그냥 heuristic인지 아닌지 확인해봐야 할 듯
close_depth, inf_depth = bds.min() * .9, bds.max() * 5.
dt = .75
mean_dz = 1. / (((1. - dt) / close_depth + dt / inf_depth))
focal = mean_dz

In [168]:
close_depth, inf_depth, (1. - dt) / close_depth, dt / inf_depth

(1.200000035762787,
 65.7046365737915,
 0.20833332712451635,
 0.011414719555714926)

In [193]:
# Get radii for spiral path
shrink_factor = .8
zdelta = close_depth * .2

# 이건 말 그대로 translation을 말하는 것으로 보인다. 즉 이미지를 찍은 camera의 pose의 translation vector
tt = poses[:, :3, 3]  # ptstocam(poses[:3,3,:].T, c2w).T
rads = np.percentile(np.abs(tt), 90, 0)
c2w_path = c2w
N_views = 120
N_rots = 2

In [194]:
tt.shape, rads

((34, 3), array([0.28753879, 0.14944784, 0.08435769]))

## Get spiral path

In [203]:
def render_path_spiral(c2w, up, rads, focal, zdelta, zrate, rots, N):
    render_poses = []
    rads = np.array(list(rads) + [1.])
    hwf = c2w[:, 4:5]

    for theta in np.linspace(0., 2. * np.pi * rots, N + 1)[:-1]:
        c = np.dot(c2w[:3, :4], np.array([np.cos(theta), -np.sin(theta), -np.sin(theta * zrate), 1.]) * rads)
        z = normalize(c - np.dot(c2w[:3, :4], np.array([0, 0, -focal, 1.])))
        render_poses.append(np.concatenate([get_viewmatrix(z, up, c), hwf], 1))
    return render_poses

In [196]:
rots=N_rots
N=N_views
zrate=.5

# Generate poses for spiral path
# render_poses = render_path_spiral(c2w_path, up, rads, focal, zdelta, zrate=.5, rots=N_rots, N=N_views)

render_poses = []
rads = np.array(list(rads) + [1.])
hwf = c2w[:, 4:5]

rads, hwf

(array([0.28753879, 0.14944784, 0.08435769, 1.        ]),
 array([[378.     ],
        [504.     ],
        [446.88232]], dtype=float32))

In [197]:
theta = 0
c2w[:3, :4].shape, np.array([np.cos(theta), -np.sin(theta), -np.sin(theta * zrate), 1.]).shape

((3, 4), (4,))

In [198]:
np.array([np.cos(theta), -np.sin(theta), -np.sin(theta * zrate), 1.]) * rads

array([ 0.28753879, -0.        , -0.        ,  1.        ])

In [207]:
render_poses = []
for theta in np.linspace(0., 2. * np.pi * rots, N + 1)[:-1]:
    # theta만큼 돌아가면서 X_{4x1}을 알려주고 이에 따라서 위치를 camera space에 올리기
    # 우선 np.array로 만드는 것은 world frame에서의 homogeneous coordi
    # 그리고 rad는 camera matrix의 translation에 해당
    # 결과적으로 x, y, z축으로 [cos, -sin, -sin(rate)]만큼 회전시켜 spiral이 나오게 되는 것
    # 그리고 이를 translation과 곱하니 원래 pixel이 출발한 위치에서 변화가 되는 것이다.
    # 이를 camera matrix(Intri * Extri)을 곱하게 되면 그 변화가 image plane에서의 좌표가 나오게 된다.
    c = np.dot(c2w[:3, :4], np.array([np.cos(theta), -np.sin(theta), -np.sin(theta * zrate), 1.]) * rads)

    # 의미 그대로 camera space상에서 z축을 말한다.
    # x,y 축 값은 0으로 없어지게 되고 z축 값만 dot product으로 남게 되니 camera coordi 에서의 z축 방향만 남음
    z = normalize(c - np.dot(c2w[:3, :4], np.array([0, 0, -focal, 1.])))
    render_poses.append(np.concatenate([get_viewmatrix(z, up, c), hwf], 1))
render_poses = np.array(render_poses).astype(np.float32)

In [208]:
render_poses.shape

(120, 3, 5)